![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


In [17]:
list = [
    "https://www.immoweb.be/en/classified/mansion/for-sale/nivelles/1400/10890514",
    "https://www.immoweb.be/en/classified/house/for-sale/hofstade/9308/10665164",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/uccle/1180/10857846",
    "https://www.immoweb.be/en/classified/villa/for-sale/zottegem/9620/10887065",
    "https://www.immoweb.be/en/classified/10708261",
    "https://www.immoweb.be/en/classified/house/for-sale/wetteren/9230/10886879",
    "https://www.immoweb.be/en/classified/mansion/for-sale/seneffe/7180/10890515",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/gent-mariakerke/9030/10890750",
    "https://www.immoweb.be/en/classified/house/for-sale/liege/4000/10886423",
    "https://www.immoweb.be/en/classified/villa/for-sale/denderleeuw-welle/9473/10887066",
    "https://www.immoweb.be/en/search/house/for_sale?countries=BE&page=333&orderBy=relevance",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/antwerp/2018/10888588",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/ingooigem/8570/10885434",
    "https://www.immoweb.be/en/classified/house/for-sale/huy/4500/10883942",
    "https://www.immoweb.be/en/classified/house/for-sale/ruisbroek/2870/10242392",
    "https://www.immoweb.be/en/classified/villa/for-sale/braine-lalleud/1420/10887894",
    "https://www.immoweb.be/en/classified/house/for-sale/gavere/9890/10880405",
    "https://www.immoweb.be/en/classified/house/for-sale/huy/4500/10888646",
    "https://www.immoweb.be/en/classified/villa/for-sale/rhode-saint-genese/1640/10887502",
    "https://www.immoweb.be/en/classified/house/for-sale/wanze/4520/10888781",
    "https://www.immoweb.be/en/classified/house/for-sale/tienen/3300/10886337",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/lierde/9570/10887816",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/gavere/9890/10890828",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/geraardsbergen-moerbeke/9500/10890849",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/wetteren/9230/10885702",
    "https://www.immoweb.be/en/search/house/for_sale?countries=BE&orderBy=relevance",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/gent-gentbrugge/9050/10885710",
    "https://www.immoweb.be/en/classified/apartment-block/for-sale/mechelen/2800/10887612",
    "https://www.immoweb.be/en/classified/house/for-sale/liege/4030/10886425",
    "https://www.immoweb.be/en/search/house/for_sale?countries=BE&page=2&orderBy=relevance",
    "https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10885478",
    "https://www.immoweb.be/en/classified/house/for-sale/leuven/3001/10886336",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/langemark-poelkapelle/8920/10892051",
    "https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/westerlo/2260/10889529",
]

for i in list:
    if "https://www.immoweb.be/en/search" in i:
        print(i)

https://www.immoweb.be/en/search/house/for_sale?countries=BE&page=333&orderBy=relevance
https://www.immoweb.be/en/search/house/for_sale?countries=BE&orderBy=relevance
https://www.immoweb.be/en/search/house/for_sale?countries=BE&page=2&orderBy=relevance


# Import data

In [1]:
import gc
from pathlib import Path
from typing import List, Optional, Tuple

import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
from data import pre_process, utils
from features import feature_engineering
from IPython.display import clear_output
from lets_plot import *
from lets_plot.mapping import as_discrete
from models import train_model
from sklearn import (
    compose,
    ensemble,
    impute,
    metrics,
    model_selection,
    neighbors,
    pipeline,
    preprocessing,
)
from tqdm.notebook import tqdm

LetsPlot.setup_html()

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


**Objective**:


1. **Cross-validation**: Implement a robust cross-validation strategy to assess our model's performance accurately and consistently across multiple folds of the data.
2. **Outlier Identification**: Identify and scrutinize potential outliers within our dataset. This step is crucial to ensure that outliers don't unduly influence our model's predictions.
3. **Feature Engineering**: Continue refining and expanding our feature engineering efforts. We'll explore new ways to create informative features that enhance our model's predictive capabilities.



# Prepare dataframe before modelling
## Read in dataframe

Drawing from our findings in notebook 4, particularly with regards to our initial feature reduction efforts, we've developed a function named "prepare_data_for_modelling." This function resides in the pre_process.py file, ensuring its reusability. The function performs essential data preprocessing steps, which include:

1. Randomly shuffling the rows in the DataFrame.
2. Transforming the 'price' column by taking the base 10 logarithm.
3. Handling missing values in categorical variables by replacing them with 'missing value.'
4. Separating the dataset into features (X) and the target variable (y).

Let's dive into the details of this function and prepare our X and y for subsequent processing.eline.


In [2]:
df = pd.read_parquet(
    utils.Configuration.INTERIM_DATA_PATH.joinpath(
        "2023-10-01_Processed_dataset_for_NB_use.parquet.gzip"
    )
)

In [3]:
def prepare_data_for_modelling(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Prepare data for machine learning modeling.

    This function takes a DataFrame and prepares it for machine learning by performing the following steps:
    1. Randomly shuffles the rows of the DataFrame.
    2. Converts the 'price' column to the base 10 logarithm.
    3. Fills missing values in categorical variables with 'missing value'.
    4. Separates the features (X) and the target (y).

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the dataset.

    Returns:
    - Tuple[pd.DataFrame, pd.Series]: A tuple containing the prepared features (X) and the target (y).

    Example use case:
    ```python
    # Load your dataset into a DataFrame (e.g., df)
    df = load_data()

    # Prepare the data for modeling
    X, y = prepare_data_for_modelling(df)

    # Now you can use X and y for machine learning tasks.
    ```
    """

    processed_df = (
        df.sample(frac=1, random_state=utils.Configuration.seed)
        .reset_index(drop=True)
        .assign(price=lambda df: np.log10(df.price))
    )

    # Fill missing categorical variables with "missing value"
    for col in processed_df.columns:
        if processed_df[col].dtype.name in ("bool", "object", "category"):
            processed_df[col] = processed_df[col].fillna("missing value")

    # Separate features (X) and target (y)
    X = processed_df.loc[:, utils.Configuration.features_to_keep]
    y = processed_df[utils.Configuration.target_col]

    print(f"Shape of X and y: {X.shape}, {y.shape}")

    return X, y

In [4]:
X, y = pre_process.prepare_data_for_modelling(df)

Shape of X and y: (3660, 16), (3660,)


# Cross-validation

Our next critical step is to establish a well-structured cross-validation strategy. This step is imperative as it enables us to assess the effectiveness of various feature engineering approaches without risking overfitting our model. A robust cross-validation strategy ensures that our model's performance evaluations are reliable and that the insights gained are generalizable to new data. To accomplish this, we will employ RepeatedKFold validation, setting the parameters with n_splits as 10 and n_repeats as 1.

In essence, this configuration signifies that we will perform a 10-fold cross-validation, and this entire process will be repeated once. Importantly, due to the modular nature of this function, we retain the flexibility to easily adapt and alter the design of our cross-validation strategy as needed.

In [5]:
def run_catboost_CV(
    X: pd.DataFrame,
    y: pd.Series,
    n_splits: int = 10,
    n_repeats: int = 1,
    pipeline: Optional[object] = None,
) -> Tuple[float, float]:
    """
    Perform Cross-Validation with CatBoost for regression.

    This function conducts Cross-Validation using CatBoost for regression tasks. It iterates
    through folds, trains CatBoost models, and computes the mean and standard deviation of the
    Root Mean Squared Error (RMSE) scores across folds.

    Parameters:
    - X (pd.DataFrame): The feature matrix.
    - y (pd.Series): The target variable.
    - n_splits (int, optional): The number of splits in K-Fold cross-validation.
      Defaults to 2.
    - n_repeats (int, optional): The number of times the K-Fold cross-validation is repeated.
      Defaults to 1.
    - pipeline (object, optional): Optional data preprocessing pipeline. If provided,
      it's applied to the data before training the model. Defaults to None.

    Returns:
    - Tuple[float, float]: A tuple containing the mean RMSE and standard deviation of RMSE
      scores across cross-validation folds.

    Example:
    ```python
    # Load your feature matrix (X) and target variable (y)
    X, y = load_data()

    # Perform Cross-Validation with CatBoost
    mean_rmse, std_rmse = run_catboost_CV(X, y, n_splits=5, n_repeats=2, pipeline=data_pipeline)

    print(f"Mean RMSE: {mean_rmse:.4f}")
    print(f"Standard Deviation of RMSE: {std_rmse:.4f}")
    ```

    Notes:
    - Ensure that the input data `X` and `y` are properly preprocessed and do not contain any
      missing values.
    - The function uses CatBoost for regression with optional data preprocessing via the `pipeline`.
    - RMSE is a common metric for regression tasks, and lower values indicate better model
      performance.
    """
    results = []

    # Extract feature names and data types
    features = X.columns[~X.columns.str.contains("price")]
    numerical_features = X.select_dtypes("number").columns.to_list()
    categorical_features = X.select_dtypes("object").columns.to_list()

    # Create a K-Fold cross-validator
    CV = model_selection.RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=utils.Configuration.seed
    )

    for train_fold_index, val_fold_index in tqdm(CV.split(X)):
        X_train_fold, X_val_fold = X.loc[train_fold_index], X.loc[val_fold_index]
        y_train_fold, y_val_fold = y.loc[train_fold_index], y.loc[val_fold_index]

        # Apply optional data preprocessing pipeline
        if pipeline is not None:
            X_train_fold = pipeline.fit_transform(X_train_fold)
            X_val_fold = pipeline.transform(X_val_fold)

        # Create CatBoost datasets
        catboost_train = Pool(
            X_train_fold,
            y_train_fold,
            cat_features=categorical_features,
        )
        catboost_valid = Pool(
            X_val_fold,
            y_val_fold,
            cat_features=categorical_features,
        )

        # Initialize and train the CatBoost model
        model = catboost.CatBoostRegressor(**utils.Configuration.catboost_params)
        model.fit(
            catboost_train,
            eval_set=[catboost_valid],
            early_stopping_rounds=utils.Configuration.early_stopping_round,
            verbose=utils.Configuration.verbose,
            use_best_model=True,
        )

        # Calculate OOF validation predictions
        valid_pred = model.predict(X_val_fold)

        RMSE_score = metrics.mean_squared_error(y_val_fold, valid_pred, squared=False)

        results.append(RMSE_score)

    return np.mean(results), np.std(results)

Now, let's proceed to train our model with the updated settings:

In [6]:
train_model.run_catboost_CV(X, y)

(0.11251233080551612, 0.004459362099695207)


::: {.callout-note}
Please note that we reduced the number of iterations here in comparison to Notebook 4, in order to reduce the training time.
NB4 :  iterations = 1000, learning rate = default == 0.03
NB5 : iterations = 100, learning rate = 0.2

Performance of baseline model : 0.1125m

:::

# Outlier detection

An outlier is a data point that stands apart from the majority of the data. One common way to define an outlier is any data point that falls more than 1.5 interquartile ranges (IQRs) below the first quartile or above the third quartile. Detecting and subsequently removing outliers from the dataset is crucial for achieving a stable model that can effectively generalize to new data.

When we create a scatter plot of our features (@fig-fig1), such as cadastral income against living area, and color the points while adjusting their size based on price, we can identify at least two data points that clearly deviate from the anticipated range of values. One of these points suggests that a 300 m2 property is associated with a cadastral income of over 320,000 EURO, while the other point is at the opposite extreme, indicating a 2,500 EUR cadastral income for an 11,000 m2 property. Both of these observations appear implausible when compared to the majority of data points on the graph.

In [7]:
# | fig-cap: "Assessing Feature Cardinality: Percentage of Unique Values per Feature"
# | label: fig-fig1

pd.concat([X, y], axis=1).pipe(
    lambda df: ggplot(
        df, aes("cadastral_income", "living_area", fill="price", size="price")
    )
    + geom_point(alpha=0.5, shape=21, stroke=0.5, show_legend=False)
    + scale_fill_continuous(low="#1a9641", high="#d7191c")
    + labs(
        title="Assessing Potential Outliers",
        subtitle=""" Outliers pose a challenge for gradient boosting methods since boosting constructs each tree based on the errors of the previous trees. 
        Outliers, having significantly larger errors than non-outliers, can excessively divert the model's attention toward these data points.
            """,
        x="Cadastral income (EUR)",
        y="Living area (m2)",
        caption="https://www.immoweb.be/",
    )
    + theme(
        plot_subtitle=element_text(
            size=12, face="italic"
        ),  # Customize subtitle appearance
        plot_title=element_text(size=15, face="bold"),  # Customize title appearance
    )
    + ggsize(800, 600)
)

To pinpoint potential outliers within our data, we can utilize Scikit-learn's LocalOutlierFactor. The Local Outlier Factor (LOF) algorithm is an unsupervised technique for anomaly detection. It calculates the local density deviation of a specific data point concerning its neighboring points. LOF identifies as outliers those data points that exhibit significantly lower density compared to their neighbors.

In the code below, we've crafted a function named "identify_outliers." This function essentially generates a mask that we can subsequently use to filter out data points that are potentially flagged as outliers.

In [8]:
def identify_outliers(df: pd.DataFrame) -> pd.Series:
    """
    Identify outliers in a DataFrame.

    This function uses a Local Outlier Factor (LOF) algorithm to identify outliers in a given
    DataFrame. It operates on both numerical and categorical features, and it returns a binary
    Series where `True` represents an outlier and `False` represents a non-outlier.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing features for outlier identification.

    Returns:
    - pd.Series: A Boolean Series indicating outliers (True) and non-outliers (False).

    Example:
    ```python
    # Load your DataFrame with features (df)
    df = load_data()

    # Identify outliers using the function
    outlier_mask = identify_outliers(df)

    # Use the outlier mask to filter your DataFrame
    filtered_df = df[~outlier_mask]  # Keep non-outliers
    ```

    Notes:
    - The function uses Local Outlier Factor (LOF) with default parameters for identifying outliers.
    - Numerical features are imputed using median values, and categorical features are one-hot encoded
      and imputed with median values.
    - The resulting Boolean Series is `True` for outliers and `False` for non-outliers.
    """
    # Extract numerical and categorical feature names
    NUMERICAL_FEATURES = df.select_dtypes("number").columns.tolist()
    CATEGORICAL_FEATURES = df.select_dtypes("object").columns.tolist()

    # Define transformers for preprocessing
    numeric_transformer = pipeline.Pipeline(
        steps=[("imputer", impute.SimpleImputer(strategy="median"))]
    )

    categorical_transformer = pipeline.Pipeline(
        steps=[
            ("encoder", preprocessing.OneHotEncoder(handle_unknown="ignore")),
            ("imputer", impute.SimpleImputer(strategy="median")),
        ]
    )

    # Create a ColumnTransformer to handle both numerical and categorical features
    preprocessor = compose.ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, NUMERICAL_FEATURES),
            ("cat", categorical_transformer, CATEGORICAL_FEATURES),
        ]
    )

    # Initialize the LocalOutlierFactor model
    clf = neighbors.LocalOutlierFactor()

    # Fit LOF to preprocessed data and make predictions
    y_pred = clf.fit_predict(preprocessor.fit_transform(df))

    # Adjust LOF predictions to create a binary outlier mask
    y_pred_adjusted = [1 if x == -1 else 0 for x in y_pred]
    outlier_mask = pd.Series(y_pred_adjusted) == 0

    return outlier_mask

As a comparison, here's the scatter plot after removing outliers. It appears that the LocalOutlierFactor method was effective in addressing the outlier data points.

In [9]:
outlier_mask = pre_process.identify_outliers(X)

X_wo_outliers = X.loc[outlier_mask, :].reset_index(drop=True)
y_wo_outliers = y.loc[outlier_mask].reset_index(drop=True)


(
    pd.concat([X_wo_outliers, y_wo_outliers], axis=1)
    # .loc[lambda df: pre_process.identify_outliers(df.loc[:, :"living_area"])]
    .pipe(
        lambda df: ggplot(
            df, aes("cadastral_income", "living_area", fill="price", size="price")
        )
        + geom_point(alpha=0.5, shape=21, stroke=0.5, show_legend=False)
        + scale_fill_continuous(low="#1a9641", high="#d7191c")
        + labs(
            title="Assessing Potential Outliers",
            subtitle=""" By employing the default parameters of LocalOutlierFactor, we've reduced our training set from 3660 instances to 3427.
            This is expected to enhance our model's performance and its ability to generalize well to new data.
            """,
            x="Cadastral income (EUR)",
            y="Living area (m2)",
            caption="https://www.immoweb.be/",
        )
        + theme(
            plot_subtitle=element_text(
                size=12, face="italic"
            ),  # Customize subtitle appearance
            plot_title=element_text(size=15, face="bold"),  # Customize title appearance
        )
        + ggsize(800, 600)
    )
)

Now, let's assess whether our efforts to improve the model by addressing outliers have enhanced its predictive capabilities:

In [10]:
train_model.run_catboost_CV(X_wo_outliers, y_wo_outliers)

(0.11052917780860605, 0.004569457889717371)

::: {.callout-note}
By removing the outliers, our cross-validation RMSE score decreased from 0.1125 to 0.1105.
:::

# Feature Engineering

**SOME IDEAS**

1. **Categorical Features:**
   - Encode categorical variables like "state," "kitchen_type," "building_condition," and "city" using one-hot encoding or label encoding.

2. **Geospatial Features:**
   - Calculate the distance from each apartment to key locations in the city (e.g., city center, schools, parks, public transportation) using "lat" and "lng."
   - Create clusters or neighborhoods based on the latitude and longitude coordinates (lat, lng) to capture the regional effect on prices.

3. **Spatial Features:**
   - Explore the "number_of_frontages" and how it affects property prices. You could categorize this variable or use it as a numerical feature.

4. **Area-related Features:**
   - Calculate the ratio of "living_area" to "surface_of_the_plot" to get an idea of the density or spaciousness of the property.
   - Create bins or categories for "living_area" and "surface_of_the_plot" to capture different property sizes.

5. **Energy Efficiency Features:**
   - Compute the energy efficiency ratio by dividing "yearly_theoretical_total_energy_consumption" by "primary_energy_consumption."
   - Normalize energy-related features to have a similar scale if they are measured in different units.

6. **Toilet and Bathroom Features:**
   - Combine "toilets" and "bathrooms" into a single "total_bathrooms" feature to simplify the model.

7. **Parking Features:**
   - If parking information is available, create binary features indicating the presence of covered or uncovered parking spaces.

8. **Taxation Features:**
   - Incorporate "cadastral_income" as a measure of property value for taxation. You can create bins or categories for this variable.

9. **Combining Features:**
   - Experiment with interactions and products of different features to capture complex relationships. For instance, you could multiply "living_area" by "cadastral_income" to get a new feature.
  
10. 
TODO : 

Feature engineering:
- use EJ categorical column for groupby/transform
- create bins from continous data and groupby/transform : sklearn.preprocessing.KBinsDiscretizer
- create column that has flags where missing values present : sklearn.impute.MissingIndicator
- interons
'''


## Use categorical columns for groupby/transform

In [11]:
# Number of unique categories per categorical variables:

X_wo_outliers.select_dtypes("object").nunique()

state                   9
kitchen_type            9
street                456
building_condition      7
city                  230
dtype: int64

In [12]:
def FE_categorical_transform(
    X: pd.DataFrame, y: pd.Series, transform_type: str = "mean"
) -> pd.DataFrame:
    """
    Feature Engineering: Transform categorical features using CatBoost Cross-Validation.

    This function performs feature engineering by transforming categorical features using CatBoost
    Cross-Validation. It calculates the mean and standard deviation of Out-Of-Fold (OOF) Root Mean
    Squared Error (RMSE) scores for various combinations of categorical and numerical features.

    Parameters:
    - X (pd.DataFrame): The input DataFrame containing both categorical and numerical features.
    - transform_type (str, optional): The transformation type, such as "mean" or other valid
      CatBoost transformations. Defaults to "mean".

    Returns:
    - pd.DataFrame: A DataFrame with columns "mean_OOFs," "std_OOFs," "categorical," and "numerical,"
      sorted by "mean_OOFs" in ascending order.

    Example:
    ```python
    # Load your DataFrame with features (X)
    X = load_data()

    # Perform feature engineering
    result_df = FE_categorical_transform(X, transform_type="mean")

    # View the DataFrame with sorted results
    print(result_df.head())
    ```

    Notes:
    - This function uses CatBoost Cross-Validation to assess the quality of transformations for
      various combinations of categorical and numerical features.
    - The resulting DataFrame provides insights into the effectiveness of different transformations.
    - Feature engineering can help improve the performance of machine learning models.
    """
    # Initialize a list to store results
    results = []

    # Get a list of categorical and numerical columns
    categorical_columns = X.select_dtypes("object").columns
    numerical_columns = X.select_dtypes("number").columns

    # Combine the loops to have a single progress bar
    for categorical in tqdm(categorical_columns, desc="Progress"):
        for numerical in tqdm(numerical_columns):
            # Create a deep copy of the input data
            temp = X.copy(deep=True)

            # Calculate the transformation for each group within the categorical column
            temp["new_column"] = temp.groupby(categorical)[numerical].transform(
                transform_type
            )

            # Run CatBoost Cross-Validation with the transformed data
            mean_OOF, std_OOF = train_model.run_catboost_CV(temp, y)

            # Store the results as a tuple
            result = (mean_OOF, std_OOF, categorical, numerical)
            results.append(result)

            del temp, mean_OOF, std_OOF

    # Create a DataFrame from the results and sort it by mean OOF scores
    result_df = pd.DataFrame(
        results, columns=["mean_OOFs", "std_OOFs", "categorical", "numerical"]
    )
    result_df = result_df.sort_values(by="mean_OOFs")
    return result_df

In [13]:
%%script echo skipping

FE_categorical_transform_mean = feature_engineering.FE_categorical_transform(
    X_wo_outliers, y_wo_outliers
)

Couldn't find program: 'echo'


In [14]:
%%script echo skipping

FE_categorical_transform_mean.to_parquet(
    f'{utils.Configuration.INTERIM_DATA_PATH.joinpath("FE_categorical_transform_mean")}.parquet.gzip',
    compression="gzip",
)

FE_categorical_transform_mean.head(15)

Couldn't find program: 'echo'


In [15]:
pd.read_parquet(
    utils.Configuration.INTERIM_DATA_PATH.joinpath(
        "FE_categorical_transform_mean.parquet.gzip"
    )
).head()

,mean_OOFs,std_OOFs,categorical,numerical
53,0.108973,0.006262,city,cadastral_income
39,0.108985,0.004980,building_condition,yearly_theoretical_total_energy_consumption
33,0.109381,0.005434,building_condition,bedrooms
31,0.109478,0.004887,street,cadastral_income
43,0.109540,0.004944,building_condition,living_area


# Create bins from continuous data and use groupby/transform

In [16]:
def FE_continuous_transform(
    X: pd.DataFrame, y: pd.Series, transform_type: str = "mean"
) -> pd.DataFrame:
    """
    Feature Engineering: Transform continuous features using CatBoost Cross-Validation.

    This function performs feature engineering by transforming continuous features using CatBoost
    Cross-Validation. It calculates the mean and standard deviation of Out-Of-Fold (OOF) Root Mean
    Squared Error (RMSE) scores for various combinations of discretized and transformed continuous
    features.

    Parameters:
    - X (pd.DataFrame): The input DataFrame containing both continuous and categorical features.
    - y (pd.Series): The target variable for prediction.
    - transform_type (str, optional): The transformation type, such as "mean" or other valid
      CatBoost transformations. Defaults to "mean".

    Returns:
    - pd.DataFrame: A DataFrame with columns "mean_OOFs," "std_OOFs," "discretized_continuous,"
      and "transformed_continuous," sorted by "mean_OOFs" in ascending order.

    Example:
    ```python
    # Load your DataFrame with features (X) and target variable (y)
    X, y = load_data()

    # Perform feature engineering
    result_df = FE_continuous_transform(X, y, transform_type="mean")

    # View the DataFrame with sorted results
    print(result_df.head())
    ```

    Notes:
    - This function uses CatBoost Cross-Validation to assess the quality of transformations for
      various combinations of discretized and transformed continuous features.
    - The number of bins for discretization is determined using Sturges' rule.
    - The resulting DataFrame provides insights into the effectiveness of different transformations.
    - Feature engineering can help improve the performance of machine learning models.
    """
    # Initialize a list to store results
    results = []

    # Get a list of continuous and numerical columns
    continuous_columns = X.select_dtypes("number").columns
    optimal_bins = int(np.floor(np.log2(X.shape[0])) + 1)

    # Combine the loops to have a single progress bar
    for discretized_continuous in tqdm(continuous_columns, desc="Progress:"):
        for transformed_continuous in tqdm(continuous_columns):
            if discretized_continuous != transformed_continuous:
                # Create a deep copy of the input data
                temp = X.copy(deep=True)

                discretizer = pipeline.Pipeline(
                    steps=[
                        ("imputer", impute.SimpleImputer(strategy="median")),
                        (
                            "add_bins",
                            preprocessing.KBinsDiscretizer(
                                encode="ordinal", n_bins=optimal_bins
                            ),
                        ),
                    ]
                )

                temp[discretized_continuous] = discretizer.fit_transform(
                    X[[discretized_continuous]]
                )

                # Calculate the transformation for each group within the categorical column
                temp["new_column"] = temp.groupby(discretized_continuous)[
                    transformed_continuous
                ].transform(transform_type)

                # Run CatBoost Cross-Validation with the transformed data
                mean_OOF, std_OOF = train_model.run_catboost_CV(temp, y)

                # Store the results as a tuple
                result = (
                    mean_OOF,
                    std_OOF,
                    discretized_continuous,
                    transformed_continuous,
                )
                results.append(result)

                del temp, mean_OOF, std_OOF

    # Create a DataFrame from the results and sort it by mean OOF scores
    result_df = pd.DataFrame(
        results,
        columns=[
            "mean_OOFs",
            "std_OOFs",
            "discretized_continuous",
            "transformed_continuous",
        ],
    )
    result_df = result_df.sort_values(by="mean_OOFs")
    return result_df

In [22]:
%%script echo skipping

FE_continuous_transform_mean = feature_engineering.FE_continuous_transform(
    X_wo_outliers, y_wo_outliers
)

FE_continuous_transform_mean.to_parquet(
    f'{utils.Configuration.INTERIM_DATA_PATH.joinpath("FE_continuous_transform_mean")}.parquet.gzip',
    compression="gzip",
)

FE_categorical_transform_mean.head(15)

Couldn't find program: 'echo'


In [27]:
pd.read_parquet(
    utils.Configuration.INTERIM_DATA_PATH.joinpath(
        "FE_continuous_transform_mean.parquet.gzip"
    )
).head(10)

,mean_OOFs,std_OOFs,discretized_continuous,transformed_continuous
55,0.109328,0.005094,bathrooms,yearly_theoretical_total_energy_consumption
59,0.109328,0.005094,bathrooms,living_area
58,0.109328,0.005094,bathrooms,cadastral_income
57,0.109328,0.005094,bathrooms,lat
56,0.109328,0.005094,bathrooms,surface_of_the_plot
50,0.109328,0.005094,bathrooms,bedrooms
52,0.109328,0.005094,bathrooms,toilets
39,0.109417,0.004831,lng,living_area
1,0.109426,0.004587,bedrooms,toilets
4,0.109426,0.004587,bedrooms,bathrooms
